In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pylab as plt
import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline
from pathlib import Path
import prettytable
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
from scipy.stats.mstats import gmean
%pylab inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
import glob 
warnings.simplefilter(action='ignore')
pio.renderers.default = 'iframe'
import ipywidgets as widgets
pyo.init_notebook_mode(connected = True)
from ipywidgets import interactive, HBox, VBox, widgets, interact
from ipywidgets import FloatSlider

Populating the interactive namespace from numpy and matplotlib


In [2]:
def summary_metrics(R,rf_annual=0.01, show=False):

    pt = prettytable.PrettyTable(['metric', 'value'])

    avg = np.mean(R)
    std_dev = np.std(R)
    rf = rf_annual / 252
    count = len(R)
    sharpe = (avg - rf) / std_dev * np.sqrt(252)

    avg_annualized_return = (((1 + avg) ** 252) - 1)
    geo_mean = gmean(R + 1) - 1
    annualized_std = std_dev * np.sqrt(252)

    cumulative_end_return = np.cumsum(R).tail(1).values

    win = len(R[R >= 0])
    loss = len(R[R < 0])
    win_loss = win / loss
    win_ratio = win / (loss + win)

    rpt = {'mean': avg,
           'avg_annualized_ret': avg_annualized_return,
           'geometric_mean': geo_mean,
           'cum_end_return': cumulative_end_return,
           'std_dev': std_dev,
           'annualized_std': annualized_std,
           'Sharpe_ratio': sharpe,
           'skewness': R.skew(),
           'kurtosis': R.kurtosis(),
           'drawdown': get_drawdown(R),
           'win_loss': win_loss,
           'win_ratio': win_ratio,
           'count': count
           }
    for (k, v) in rpt.items():
        pt.add_row([k, v])

    if show:
        print(pt)

    return rpt

In [3]:
def get_drawdown(R):
    # Calculated the peak
    roll_max = R.max()
    # Calculated difference of daily returns from the peak
    daily_drawdown = R/roll_max - 1.0
    # Found the maximum drop
    max_daily_drawdown = daily_drawdown.min()
    return max_daily_drawdown

In [4]:
def compare_low(rpt_his_low,rpt_hat_low):
    table_low = prettytable.PrettyTable(['metric', 'covar_his_low', 'covar_MGARCH_low'])
    for (k, v) in rpt_his_low.items():
        row = [k, v, rpt_hat_low[k]]
        table_low.add_row(row)
    print(table_low)
    return table_low

def compare_med(rpt_his_med,rpt_hat_med):
    table_med = prettytable.PrettyTable(['metric', 'covar_his_med', 'covar_MGARCH_med'])
    for (k, v) in rpt_his_med.items():
        row = [k, v, rpt_hat_med[k]]
        table_med.add_row(row)
    print(table_med)
    return table_med

def compare_high(rpt_his_high,rpt_hat_high):
    table_high = prettytable.PrettyTable(['metric', 'covar_his_high', 'covar_MGARCH_high'])
    for (k, v) in rpt_his_high.items():
        row = [k, v, rpt_hat_high[k]]
        table_high.add_row(row)
    print(table_high)
    return table_high

In [5]:
filenametab = []
for filename in glob.glob('../outputdata/Stage2/Returns/*.csv'):
    filenametab.append(filename)
filenametab.sort(reverse=True)

returns = pd.read_csv(filenametab[0], parse_dates=True)
returns['date']= pd.to_datetime(returns.date)
returns.set_index('date', inplace=True)

In [6]:
filenametab[0]

'../outputdata/Stage2/Returns\\SapiatStage2OutStage2OutputReturns20201012111638_504.csv'

In [7]:
initial_investment = 100.00

In [8]:
def apply_cumsum(returns_series):
    cumsumret = (returns_series.add(1).cumprod()) * initial_investment
    cumsumret.iat[0] = initial_investment
    return cumsumret


In [9]:
returns['ret_low_his_cum'] = apply_cumsum(returns.ret_low_his)
returns['ret_low_hat_cum'] = apply_cumsum(returns.ret_low_hat)

returns['ret_med_his_cum'] = apply_cumsum(returns.ret_med_his)
returns['ret_med_hat_cum'] = apply_cumsum(returns.ret_med_hat)

returns['ret_high_his_cum'] = apply_cumsum(returns.ret_med_his)
returns['ret_high_hat_cum'] = apply_cumsum(returns.ret_med_hat)

In [10]:
print(returns[['ret_low_his_cum','ret_low_his']])

            ret_low_his_cum  ret_low_his
date                                    
2010-04-01       100.000000    -0.000306
2010-04-05       100.115666     0.001463
2010-04-06       100.374905     0.002589
2010-04-07       100.511432     0.001360
2010-04-08       100.629635     0.001176
...                     ...          ...
2020-03-25       138.885678     0.001508
2020-03-26       141.365299     0.017854
2020-03-27       139.546228    -0.012868
2020-03-30       140.792106     0.008928
2020-03-31       139.959188    -0.005916

[2479 rows x 2 columns]


In [11]:
compare_low(summary_metrics(returns.ret_low_his),summary_metrics(returns.ret_low_hat));

+--------------------+------------------------+------------------------+
|       metric       |     covar_his_low      |    covar_MGARCH_low    |
+--------------------+------------------------+------------------------+
|        mean        | 0.00014194283915847452 | 0.00015056112332552965 |
| avg_annualized_ret |  0.03641439298404303   |  0.03866740180834349   |
|   geometric_mean   | 0.0001356206033298335  | 0.00014037143504674887 |
|   cum_end_return   |      [0.3518763]       |      [0.37324102]      |
|      std_dev       |  0.00355269285450384   |  0.004506496688490014  |
|   annualized_std   |  0.05639725066568003   |  0.07153841713128414   |
|    Sharpe_ratio    |   0.4569299950576741   |  0.39057899515384364   |
|      skewness      |  -0.7526061551855804   |  -1.0779547970318388   |
|      kurtosis      |   24.31800278423236    |   36.68680023298243    |
|      drawdown      |  -2.1660284193108614   |   -2.199158605178767   |
|      win_loss      |   1.1519097222222223   |   1

In [12]:
# final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('ret_')]]
final_csv_output_plt = returns
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
    go.Scatter(x=filtered_df.index, y = filtered_df.ret_low_hat_cum,mode = 'lines',name = 'Low Risk MGARCH Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_low_his_cum,mode = 'lines',name = 'Low Risk Historical Return'),
        
#     go.Scatter(x=filtered_df.index, y = np.cumsum(filtered_df.ret_low_hat),mode = 'lines',name = 'Low Risk MGARCH Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_low_his),mode = 'lines',name = 'Low Risk Historical Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.equity_daily_ret),mode = 'lines',name = 'Daily Equity Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.bond_daily_ret),mode = 'lines',name = 'Daily Bond Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.gold_daily_ret),mode = 'lines',name = 'Daily Gold Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.forex_daily_ret),mode = 'lines',name = 'Daily Forex Return'),
    ],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Return in USD'),title = 'Low(0.002) Risk Portfolio(S&P500,Fed T-bonds,Gold&Forex(USD)) MGARCH/Historical Return from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2012.5, 2017.5), description='Year', layout=Layout(width='900px'…

In [13]:
compare_med(summary_metrics(returns.ret_med_his),summary_metrics(returns.ret_med_hat));

+--------------------+------------------------+------------------------+
|       metric       |     covar_his_med      |    covar_MGARCH_med    |
+--------------------+------------------------+------------------------+
|        mean        | 0.00021757248803828155 | 0.00021675715917360364 |
| avg_annualized_ret |  0.05635288756339918   |  0.05613591567253762   |
|   geometric_mean   | 0.0002003100117842216  | 0.00018786171089524295 |
|   cum_end_return   |      [0.5393622]       |       [0.537341]       |
|      std_dev       |  0.005865781029890114  |  0.007576620374798872  |
|   annualized_std   |  0.09311638710149744   |  0.12027511974037683   |
|    Sharpe_ratio    |  0.48142188911156814   |  0.37100610839606646   |
|      skewness      |  -0.8120991068963196   |  -1.0936810616555874   |
|      kurtosis      |   23.864857464212264   |   45.382245062181106   |
|      drawdown      |   -2.134214859098237   |   -2.246285959674413   |
|      win_loss      |   1.1726555652936022   |   1

In [14]:
final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('ret_')]]
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
       
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_med_hat),mode = 'lines',name = 'Medium Risk MGARCH Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_med_his),mode = 'lines',name = 'Medium Risk Historical Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_med_hat_cum,mode = 'lines',name = 'Medium Risk MGARCH Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_med_his_cum,mode = 'lines',name = 'Medium Risk Historical Return'),

    ],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Return in USD'),title = 'Medium(0.006) Risk Portfolio(S&P500,Fed T-bonds,Gold&Forex(USD)) MGARCH/Historical Return from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2012.5, 2017.5), description='Year', layout=Layout(width='900px'…

In [15]:
compare_high(summary_metrics(returns.ret_high_his),summary_metrics(returns.ret_high_hat));

+--------------------+-----------------------+------------------------+
|       metric       |     covar_his_high    |   covar_MGARCH_high    |
+--------------------+-----------------------+------------------------+
|        mean        | 0.0002471503017070321 | 0.00023170901649867943 |
| avg_annualized_ret |  0.06425410277647581  |  0.060121907280793474  |
|   geometric_mean   | 0.0002151671656414056 | 0.00019161098420528155 |
|   cum_end_return   |      [0.6126856]      |      [0.57440665]      |
|      std_dev       |  0.007978264215074788 |  0.008926015658775072  |
|   annualized_std   |  0.12665101804232298  |  0.14169610579072328   |
|    Sharpe_ratio    |   0.4128026512404429  |  0.34151024749499725   |
|      skewness      |  -0.8019179144950634  |  -0.8874874062692555   |
|      kurtosis      |   25.413564313173033  |    34.2705342724524    |
|      drawdown      |  -2.2107484449700716  |  -2.2772361812279316   |
|      win_loss      |   1.1669580419580419  |   1.1707530647985

In [16]:
final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('ret_')]]
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_high_hat),mode = 'lines',name = 'High Risk MGARCH Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_high_his),mode = 'lines',name = 'High Risk Historical Return')
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_high_hat_cum,mode = 'lines',name = 'High Risk MGARCH Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_high_his_cum,mode = 'lines',name = 'High Risk Historical Return')
   ],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Return in USD'),title = 'High(0.01) Risk Portfolio(S&P500, US Fed T-bonds, Gold & Forex(USD)) MGARCH/Historical Return from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2012.5, 2017.5), description='Year', layout=Layout(width='900px'…

## Potential Metrics

- Compounded daily return charts
- Geometric returns --> Annualized return/Total return
- Compounded geometric return charts
- Win ratio
- Drawdown

## Compounded daily return <li> https://stackoverflow.com/questions/5515021/compute-a-compounded-return-series-in-python </li>

In [13]:
returns['comp_low_ret_hat'] = 0.00
returns['comp_med_ret_hat'] = 0.00
returns['comp_high_ret_hat'] = 0.00
returns['comp_low_ret_hat'].iloc[0] = returns['ret_low_hat'].iloc[0]
returns['comp_med_ret_hat'].iloc[0] = returns['ret_med_hat'].iloc[0]
returns['comp_high_ret_hat'].iloc[0] = returns['ret_high_hat'].iloc[0]
for i in range(1, len(returns)):
    returns['comp_low_ret_hat'].iloc[i] = (1 + returns['comp_low_ret_hat'].iloc[i-1]) * (1 + returns['ret_low_hat'].iloc[i]) - 1
    returns['comp_med_ret_hat'].iloc[i] = (1 + returns['comp_med_ret_hat'].iloc[i-1]) * (1 + returns['ret_med_hat'].iloc[i]) - 1
    returns['comp_high_ret_hat'].iloc[i] = (1 + returns['comp_high_ret_hat'].iloc[i-1]) * (1 + returns['ret_high_hat'].iloc[i]) - 1

In [14]:
final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('comp_')]]
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
    go.Scatter(x=filtered_df.index, y = np.cumsum(filtered_df.comp_low_ret_hat),mode = 'lines',name = 'Low Risk'),      
    go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.comp_med_ret_hat),mode = 'lines',name = 'Medium Risk'),
    go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.comp_high_ret_hat),mode = 'lines',name = 'High Risk'),        
],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Returns'),title = 'Compounded Daily Return Forecast from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2010.25, 2014.75), description='Year', layout=Layout(width='900p…

## Win Ratio -> https://www.investopedia.com/terms/w/win-loss-ratio.asp

In [15]:
# final_csv_output_plt = returns[[col for col in returns.columns if col.endswith('comp_')]]
returns_temp = returns.copy()
returns_temp['year'] = returns_temp.index.year
winratio = pd.DataFrame()
winratio['year'] = returns_temp.year.unique()
winratio['winration_low'] = 0.00
winratio['winration_med'] = 0.00
winratio['winration_high'] = 0.00
for i,year in enumerate(returns_temp.year.unique()):
    subpos = returns[(returns_temp['year'] == year)]
    positive_n = sum(n > 0 for n in returns_temp.ret_low_hat.values.flatten())
    negative_n = sum(n < 0 for n in returns_temp.ret_low_hat.values.flatten())
    winratio['winration_low'].iloc[i] = positive_n / negative_n
    
    positive_n = sum(n > 0 for n in returns_temp.ret_med_hat.values.flatten())
    negative_n = sum(n < 0 for n in returns_temp.ret_med_hat.values.flatten())
    winratio['winration_med'].iloc[i] = positive_n / negative_n
    
    positive_n = sum(n > 0 for n in returns_temp.ret_high_hat.values.flatten())
    negative_n = sum(n < 0 for n in returns_temp.ret_high_hat.values.flatten())
    winratio['winration_high'].iloc[i] = positive_n / negative_n
    
# winratio.set_index('year', inplace=True)

In [16]:
xmin = winratio.year.unique().min()
xmax = winratio.year.unique().max()
# winratio.set_index('year', inplace=True)
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = winratio.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
    go.Scatter(x=filtered_df.index, y = filtered_df.winration_low,mode = 'lines',name = 'Low Risk'),      
    go.Scatter(x=filtered_df.index,y = filtered_df.winration_med,mode = 'lines',name = 'Medium Risk'),
    go.Scatter(x=filtered_df.index,y = filtered_df.winration_high,mode = 'lines',name = 'High Risk'),        
],
    layout=go.Layout(xaxis = dict(title = 'quarter'), yaxis = dict(title = 'Win Ratio'),title = 'Win Ratio on Return Forecast from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2010.25, 2014.75), description='Year', layout=Layout(width='900p…

## Quarterly Drowdown ration https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp

In [17]:
returns_temp1 = returns[[col for col in returns.columns if col.endswith('_hat')]]
returns_temp1['quarter'] = returns_temp1.index.year.astype(str) + '-Q' + returns_temp1.index.quarter.astype(str)
# returns_temp1['quarter'] = returns_temp1.index.year.astype(str) + returns_temp1.index.quarter.astype(str)
report_q = []
for qtr in returns_temp1.quarter.unique():
    
    subpos = returns[(returns_temp1['quarter'] == qtr)]
    
    Troughvalue = min(subpos.ret_low_hat)
    Peakvalue   = max(subpos.ret_low_hat)
    mdd_qtr_low = (Troughvalue - Peakvalue)/Peakvalue #Do I need to take absolute Values?

    Troughvalue = min(subpos.ret_med_hat)
    Peakvalue   = max(subpos.ret_med_hat)
    mdd_qtr_med = (Troughvalue - Peakvalue)/Peakvalue #Do I need to take absolute Values?    

    Troughvalue = min(subpos.ret_high_hat)
    Peakvalue   = max(subpos.ret_high_hat)
    mdd_qtr_high = (Troughvalue - Peakvalue)/Peakvalue #Do I need to take absolute Values?

    r = { 'quarter': qtr,
          'mdd_qtr_low': mdd_qtr_low,
          'mdd_qtr_med': mdd_qtr_med,
          'mdd_qtr_high': mdd_qtr_high}
    report_q += [r]
df_mdd = pd.DataFrame(report_q)

In [18]:
df_mdd

,quarter,mdd_qtr_low,mdd_qtr_med,mdd_qtr_high
0,2008-Q2,-2.574150,-2.582851,-2.576738
1,2008-Q3,-1.803724,-1.770639,-1.770639
2,2008-Q4,-1.574708,-2.038997,-2.038997
3,2009-Q1,-2.069594,-1.680610,-1.606693
4,2009-Q2,-2.487110,-2.221276,-2.329960
5,2009-Q3,-1.973372,-1.973867,-1.969473
6,2009-Q4,-1.833749,-1.838996,-2.186136
7,2010-Q1,-2.109994,-2.199983,-2.732574
8,2010-Q2,-2.086429,-1.912750,-1.912750
9,2010-Q3,-1.827286,-2.429005,-2.509590


In [19]:
xmin = df_mdd.quarter.unique().min()
xmax = df_mdd.quarter.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True
#     readout_format='d',
#     orientation='horizontal',
#     description='quarter'
)
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = df_mdd.query( 'quarter>= ' + str(y[0]) +'and quarter<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
    go.Scatter(x=filtered_df.quarter, y = filtered_df.mdd_qtr_low,mode = 'lines',name = 'Low Risk'),      
    go.Scatter(x=filtered_df.quarter,y = filtered_df.mdd_qtr_med,mode = 'lines',name = 'Medium Risk'),
    go.Scatter(x=filtered_df.quarter,y = filtered_df.mdd_qtr_high,mode = 'lines',name = 'High Risk'),        
],
    layout=go.Layout(xaxis = dict(title = 'quarter'), yaxis = dict(title = 'Win Ratio'),title = 'Win Ratio on Return Forecast from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

TypeError: can't multiply sequence by non-int of type 'float'